# Predicting house prices in Ames, Iowa

The purpose of this notebook is for me to learn something about Python programming, how to process data using Python, and how to perform machine learning tasks in Python. It may perhaps also provide some decent predictions of house prices in Ames, IA.

This data holds quite a bit of interest for me, because my wife and I are currently going through the process on both sides of the table - we've put our current house on the market, and we have a new house under contract.

I'd also like to invite anyone who takes the time to read this to point out ways in which my methods or code could be better, as I'm still figuring all this out. Also, this is a work in progress, so it will have many updates.


## 1 Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.ticker as tick
import seaborn as sb
from sklearn import linear_model
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# 2 Read in the data

In [ ]:
train = pd.read_csv("../input/train.csv", header=0)
test = pd.read_csv("../input/test.csv", header=0)

## 3 Data cleaning

Let's check to what extent we need to clean up the data before throwing a model at it. On a previous version I had my own definition of these, but the results ended up being terrible, so I shamelessly stole large portions from [this kernel](https://www.kaggle.com/mountaindata/house-prices-advanced-regression-techniques/house-price-regression).

### 3.1 Get rid of NaNs
It looks like several columns have a lot of missing values. From the data dictionary provided, it seems that this should be expected. These should be interpreted as the absence of a given feature rather than a missing value. However, we're going to have to do something with them, because scikit-learn doesn't like them. We'll convert them to 0s.

In [ ]:
train = train.fillna(0)
test = test.fillna(0)

### 3.2 Separating types of features
We want to get lists of all the features - one grand list, and also separate lists for categorical and numerical features. The former we will convert to Pandas's category type, and the latter we will leave alone. We won't carry the Id column (which is useless except as an identifier) and the sale price (the target). The middle set I've identified as "ordered categories", and I'll code them myself in what I feel is a natural way.

In [ ]:
features = [x for x in train.columns if x not in ['id','SalePrice']]

In [ ]:
cat_features = ['MSSubClass', 'MSZoning', 'Street','Alley', 
'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
'HouseStyle','OverallQual','OverallCond',
'Foundation', 'BsmtFinType1','BsmtFinType2', 
'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType',
'Heating','CentralAir','Electrical','BsmtFullBath',
'BsmtHalfBath','FullBath','HalfBath','BedroomAbvGr','KitchenAbvGr',
'TotRmsAbvGrd','Functional','Fireplaces','GarageType',
'GarageFinish','GarageCars','PavedDrive',
'Fence','MiscFeature','SaleType','SaleCondition']

In [ ]:
ord_cat_features = ['ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure',
                   'HeatingQC','KitchenQual','FireplaceQu','GarageQual','GarageCond','PoolQC']

In [ ]:
num_features = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2',
'BsmtUnfSF','TotalBsmtSF', 'LowQualFinSF','1stFlrSF','2ndFlrSF',
'GrLivArea', 'YearRemodAdd', 'YearBuilt','GarageYrBlt','GarageArea','WoodDeckSF',
'OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal','MoSold','YrSold']

### 3.3 Converting categorical values to numeric values
We need to get rid of all the string values and replace them with numeric values. For this we'll lean on Pandas's categorical type, which didn't exist the last time I messed with Python extensively. They're a bit like R's "factors", which come in handy.

In [ ]:
rows_train = len(train)
rows_test = len(test)

We'll now glue the training and test set together, for all but the sale price, which the test set doesn't have in this case. Later, we'll split them back apart. We do this because the test data might have instances within a category not seen in the test data, in which case Pandas won't pick them up. For example, if there are no values of 'I' in the 'MSZoning' field in the training data but not the test data, they'll be ignored when we convert the categories to codes. Gluing the datasets together prevents this. It wouldn't help at all if you had to run the model against data you didn't have access to (which is a possibility in this case).

Another thing that annoys me is that some of these categoricals could justifiably be considered to be ordered, in which case I want the order to be preserved. Pandas won't figure this out automatically. Some are easy, like OverallQual - which is already numbered from 1-10. Nothing more needs to be done. Others have 'excellent', 'good', etc., and we'd like these to mean something as well.

For predictions - it may not matter - it's possible to just assign a number to each category, regardless of order, and move along our merry ways. However, it's important in business to be able to interpret and explain model results, and you might not get to be there to walk through your arbitrary encoding scheme with The Powers That Be(tm).

In [ ]:
def transform_nb(x):
    if x in ("NoRidge", "NridgHt", "StoneBr","Timber","Veenker"):
        return "Hi"
    elif x in ("Mitchel","OldTown","BrkSide","Sawyer","NAmes","IDOTRR","MeadowV","Edwards","NPkVill","BrDale","Blueste"):
        return "Low"
    else:
        return "Mid"

In [ ]:
# glue data sets together
train_test = pd.concat((train[features], test[features])).reset_index(drop=True)
# Convert categoricals to codes
for c in range(len(cat_features)):
    train_test[cat_features[c]] = train_test[cat_features[c]].astype('category').cat.codes

First the coding helper function, stolen from [here](https://www.analyticsvidhya.com/blog/2016/01/12-pandas-techniques-python-data-manipulation/)

In [ ]:
#Define a generic function using Pandas replace function
def coding(col, codeDict):
  colCoded = pd.Series(col, copy=True)
  for key, value in codeDict.items():
    colCoded.replace(key, value, inplace=True)
  return colCoded

Now the coding:

In [ ]:
train_test['ExterQual'] = coding(train_test['ExterQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['ExterCond'] = coding(train_test['ExterCond'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['BsmtQual'] = coding(train_test['BsmtQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['BsmtCond'] = coding(train_test['BsmtCond'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['BsmtExposure'] = coding(train_test['BsmtExposure'], {'Gd':4,'Av':3,'Mn':2,'No':1, 0:0})

In [ ]:
train_test['HeatingQC'] = coding(train_test['HeatingQC'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['KitchenQual'] = coding(train_test['KitchenQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1})
train_test['FireplaceQu'] = coding(train_test['FireplaceQu'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['GarageQual'] = coding(train_test['GarageQual'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['GarageCond'] = coding(train_test['GarageCond'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})
train_test['PoolQC'] = coding(train_test['PoolQC'], {'Ex':5, 'Gd':4,'TA':3,'Fa':2,'Po':1, 0:0})

### 3.4 Restore training and test sets
I keep forgetting about iloc. It makes cutting up Pandas DataFrames work kind of like dataframes in R do. We'll put the training and test sets back in the same boxes as before.

In [ ]:
trainX = train_test.iloc[:rows_train,:]
testX = train_test.iloc[rows_train:,:]

## 4 Data exploration
Now it's time to look at the data and see what, if anything, looks weird or notable.

### 4.1 Distribution of target variable
Let's look at the distribution of the target variable to see if we need to adjust it. Since RMSE of the logs is the standard by which we'll be measured, we'll probably have to log-transform the sale prices.

In [ ]:
fig = plt.figure()
ax = plt.axes()
n,bins,patches=plt.hist(train['SalePrice'], 30, facecolor='dimgrey')
plt.xlabel("Sale price (000s)")
vals=ax.get_xticks()
ax.set_xticklabels(['${:,.0f}'.format(x/1000) for x in vals])
plt.show()

In [ ]:
target = np.log(train['SalePrice'])

In [ ]:
fig = plt.figure()
ax = plt.axes()
n,bins,patches=plt.hist(target, 30, facecolor='dimgrey')
plt.xlabel("Log(Sale price)")
plt.show()

That makes the sale prices look more normally distributed, so it might behave more amenably to models

### 4.2 Exploring categorical features
Let's take a look at the categorical features. First, a helper function to enable my laziness.

In [ ]:
def doPlots(x, data, ii, fun):
    fig, axes = plt.subplots(len(ii) // 2, ncols = 2)
    fig.tight_layout()
    for i in range(len(ii)):
        fun(x=x[ii[i]], data=data, ax=axes[i // 2, i % 2], color='dimgrey')
    plt.show()

In [ ]:
doPlots(cat_features, train, range(0,8), sb.countplot)

In [ ]:
doPlots(cat_features, train, range(8,16), sb.countplot)

In [ ]:
doPlots(cat_features, train, range(16,24), sb.countplot)

In [ ]:
doPlots(cat_features, train, range(24,32), sb.countplot)

In [ ]:
doPlots(cat_features, train, range(32,40), sb.countplot)

In [ ]:
doPlots(cat_features, train, range(40,44), sb.countplot)

**Observations:**

* Most houses have been built since 1946. That makes sense - as that corresponds with the beginning of the post-war housing boom in the Midwest.
* Almost all are zoned for residential, are on paved streets, and do not have access to an alley.
* Most are on regular or slightly irregular lots (although degrees of irregularity aren't defined).
* Most are on level ground and have access to all public utilities.
* Few were on cul-de-sacs.
* Very few were close to arterial or feeder streets, which seems surprising.
* Almost all are single family homes of one or two stories. 
* I assume the overall quality and condition assessments were subjective. If so, people seemed a lot more ambivalent about house condition than materials quality.
* Most houses had either poured concrete or cinder block foundations. I feel like every cinder block or brick foundation I've ever seen has been associated with leaking or buckling. My wife observed that these are associated with older houses, and may actually be better because they shift with the earth, whereas concrete foundations just crack. Furthermore, the latter tend to be older than the former:

In [ ]:
np.average(train['YearBuilt'][train['Foundation']=='PConc'])

In [ ]:
np.average(train['YearBuilt'][train['Foundation']=='CBlock'])

### 4.3 Exploring numeric features
Let's look at histograms of the numerical features to see what we can see.

In [ ]:
def doHistPlots(x, data, ii, fun):
    fig, axes = plt.subplots(len(ii) // 2, ncols = 2)
    fig.tight_layout()
    for i in range(len(ii)):
        fun(data[x[ii[i]]], color='b', ax=axes[i // 2, i % 2], kde=False)
    plt.show()

In [ ]:
doHistPlots(x=num_features, data=train, ii=range(0,6), fun=sb.distplot)

In [ ]:
doHistPlots(x=num_features, data=train, ii=range(6,12), fun=sb.distplot)

In [ ]:
doHistPlots(x=num_features, data=train, ii=range(12,18), fun=sb.distplot)

In [ ]:
doHistPlots(x=num_features, data=train, ii=range(18,24), fun=sb.distplot)

**Observations:**

 - There are a small number of houses with humongous (by comparison) lots. Similarly, there appears to be a house with a 6,000 square foot basement.
 - There are houses with a lot frontage / area of 0, which makes no
   sense.
 - Other than the month of sale, ages, and sizes, none of these look very useful.

In [ ]:
train[train['TotalBsmtSF']>4000]

In [ ]:
print(train['TotalBsmtSF'].iloc[1298,])
print(train['GrLivArea'].iloc[1298,])
print(train['OverallCond'].iloc[1298,])
print(train['OverallQual'].iloc[1298,])

So, that's a really big house that sold for $160,000, was a 10 in quality, has a 480 square foot pool, and was a 5 in condition. And it's a football field's distance from the street. That looks weird, so we're just going to cut that one out of the training set.

In [ ]:
trainX = trainX.drop(train.index[[1298]])

## 5 Modeling
Now let's throw some models at what we've got and see how we do.

### 5.1 Features to use
Let's select the features we're going to use. There were a couple of columns I wanted to look at in more detail.

**Neigborhood:** There are quite a few neighborhoods. Surely these can be reduced to a few classes.

In [ ]:
plt.rcParams['figure.figsize'] = (8.75, 7.0)
ax = plt.axes()
plot1 = sb.boxplot(data=train, x='Neighborhood', y='SalePrice')
ax.set_title("Price distribution by neighborhood")
sb.despine(offset=10, trim=True)
plt.xticks(rotation=90)
plt.show()

Only one has a median sale price clearly below $100,000. It looks like there is another class with medians between $100k and $150k, another class with medians between $150k and $200k, one for medians between $200k and $250k, and finally a class for medians above $250k.

In [ ]:
def transform_nb(x):
    if x in ("NoRidge", "NridgHt", "StoneBr"):
        return 5 #Over 250
    elif x in ('CollgCr','Veenker','Crawfor','Somerst','Timber','ClearCr'):
        return 4 #200-250
    elif x in ('Mitchel','NWAmes','SawyerW','Gilbert','Blmngtn','SWISU', 'Blueste'):
        return 3 #150-200
    elif x in ('OldTown','BrkSide','Sawyer','NAmes','IDOTRR','Edwards','BrDale', 'NPkVill'):
        return 2 #100 - 150
    elif x in ('MeadowV'):
        return 1
    else:
        return 9 # Catch mistakes

In [ ]:
train['NbdClass'] = train['Neighborhood'].apply(transform_nb)
test['NbdClass'] = test['Neighborhood'].apply(transform_nb)
trainX['NbdClass'] = train['NbdClass']
# This is messed up and gives a warning, I couldn't figure out how to make it go away
z = test.loc[:,'NbdClass']
z=np.asarray(z)
testX.loc[:,'NbdClass'] = z

Let's redo the sale price plot with our new class and see how it comes out.

In [ ]:
trainX[trainX['NbdClass']==9]

In [ ]:
plt.rcParams['figure.figsize'] = (8.75, 7.0)
ax = plt.axes()
plot1 = sb.boxplot(data=train, x='NbdClass', y='SalePrice')
ax.set_title("Price distribution by neighborhood")
sb.despine(offset=10, trim=True)
plt.xticks(rotation=90)
plt.show()

That looks halfway decent. Now let's make a list of features to use.

In [ ]:
use_features = ['OverallQual','YearBuilt','YearRemodAdd','MasVnrArea','GrLivArea','TotalBsmtSF','FullBath','Fireplaces',
           'GarageYrBlt','GarageCars','NbdClass','HouseStyle','Alley','LotShape','LotConfig','Exterior1st',
           'Exterior2nd','ExterQual','ExterCond','Foundation','BsmtExposure','BsmtFinType1','HeatingQC','CentralAir',
           'FireplaceQu','GarageType','GarageFinish']

### 5.2 Lasso

In [ ]:
# Test shrinkage parameters in cross-validation
alphas = np.logspace(-10,-1,num=20)

In [ ]:
# Kick out that one weird house
trainY = train['SalePrice']
trainY = trainY.drop(train.index[[1298]])
target = np.log(trainY)

In [ ]:
lasso = linear_model.LassoCV(alphas=alphas, tol=0.0001, selection='random', random_state=17, max_iter=1000)
lasso.fit(trainX[use_features], target)

In [ ]:
print("Best alpha is",lasso.alpha_)

In [ ]:
lasso.pred = lasso.predict(trainX[use_features])
print("Competition training RMSE:", np.sqrt(np.sum(lasso.pred - target)**2))

In [ ]:
# Look at what did and didn't get squeezed out
indices = np.arange(len(trainX[use_features].columns))
indices_nz = np.nonzero(lasso.coef_)
indices_z = np.setdiff1d(indices, indices_nz)
set([use_features[i] for i in indices_z])

Whoops - It didn't really reduce any from the ones I selected. Well, let's try it anyway and see how it comes out. 

In [ ]:
# see most important and least important features
tmp = [use_features[i] for i in indices_nz[0]]
tmp = np.array(tmp)
tmp2 = lasso.coef_[lasso.coef_ != 0]
lasso.c = pd.DataFrame({'cols':tmp, 'coefs':tmp2})
lasso.c = lasso.c.sort_values(by='coefs',ascending=False)

In [ ]:
lasso.c[:5]

In [ ]:
lasso.c[-5:]

In [ ]:
sollasso = pd.DataFrame({'Id':test['Id'], 'SalePrice':np.exp(lasso.predict(testX[use_features]))})

### 5.3 Random forest
I'm going to do 10-fold cross-validation to select the best number of trees and depth of each tree in the forest.

In [ ]:
ntrees = np.arange(100,1001,100)
depths = np.arange(1,8)

In [ ]:
# Warning: This takes a bit
scores=[]
X = trainX[use_features]
Y = target
for n in ntrees:
    for d in depths:
        run_tot = 0
        for k in np.arange(0,10):
            
            # Get the kth fold of data
            testXcv = X[k*146:k*146+146]
            trainX_left = X[:k*146]
            trainX_right = X[k*146+146:]
            trainXcv = pd.concat([trainX_left, trainX_right])
            
            testYcv = Y[k*146:k*146+146]
            trainY_left = Y[:k*146]
            trainY_right = Y[k*146+146:]
            trainYcv = np.concatenate([trainY_left, trainY_right])
            
            # Fit a model and make predictions using the kth fold
            rf = RandomForestRegressor(n_estimators=n, max_depth=d, n_jobs=-1)
            rf.fit(trainXcv, trainYcv)
            preds = rf.predict(testXcv)
            
            # Add this fold's score to the previous ones
            run_tot = run_tot + np.sqrt(np.sum((preds - testYcv)**2))
            
        # Now we load the scores table with the parameters we're testing and the 10-fold average score
        scores.append({'ntrees':n, 'depth':d, 'score':run_tot/10.0})
        # Tell me where we're at - Comment this out if you find it annoying
    print("ntrees =",n)
scoredf = pd.DataFrame(scores)
print("Done!")

Now, we'll plot our results. Number of trees will be along the x-axis, score along the y-axis, and the line color will be tied to the tree depth.

In [ ]:
fig=plt.figure()
ax = plt.axes()
for d in depths:
    x = scoredf['ntrees'][scoredf['depth']==d]
    y = scoredf['score'][scoredf['depth']==d]
    plt.plot(x,y, label='Depth ' + str(d))
plt.legend(loc=9, ncol=len(depths)//2) # upper center
plt.xlabel("Number of trees")
plt.ylabel("Competition RMSE")
ax.set_title("Random forest - Competition RMSE by number of trees and depth")
plt.show()

In [ ]:
rf_best = RandomForestRegressor(n_jobs=-1, n_estimators=1000, max_depth=7, random_state=17)
rf_best.fit(trainX[use_features], target)

One of the cool things about random forests is that you can get an assessment of which features contributed the most to the predictions. This is a good check of common sense. In this version they add up to 100%.

In [ ]:
importances = pd.DataFrame({'Feature':trainX[use_features].columns, 'Importance':rf_best.feature_importances_})
importances = importances.sort_values('Importance',ascending=False).set_index('Feature')
importances[0:10].iloc[::-1].plot(kind='barh',legend=False)
plt.show()

So the model at least makes some logical sense. Let's see how it did:

In [ ]:
rf.preds = np.exp(rf_best.predict(testX[use_features]))
solrf = pd.DataFrame({'Id':test['Id'], 'SalePrice':rf.preds})
solrf.to_csv("./solrf2.csv", index=False)
solrf.head(3)

### 5.4 Random forest - full dataset
We'll try it with everything but the kitchen sink.

In [ ]:
rf_all = RandomForestRegressor(n_jobs=-1, n_estimators=1000, max_depth=7, random_state=17)
rf_all.fit(trainX, target)

In [ ]:
importances = pd.DataFrame({'Feature':trainX.columns, 'Importance':rf_all.feature_importances_})
importances = importances.sort_values('Importance',ascending=False).set_index('Feature')
importances[0:10].iloc[::-1].plot(kind='barh',legend=False)
plt.show()

In [ ]:
rf.preds = np.exp(rf_all.predict(testX))
solrf = pd.DataFrame({'Id':test['Id'], 'SalePrice':rf.preds})
solrf.to_csv("./solrf-full.csv", index=False)
solrf.head(3)

**Results:** The two random forest models scored at 0.15453 and 0.15129 - neither of which beat the "lasso" model. I put the lasso in quotes because it didn't actually squelch out any of the columns, but hey - if it's stupid but it works, it's not stupid.

### 5.5 XGBoost
This is the one the competition description said to try, so I'm trying it.

In [ ]:
train_X = trainX.iloc[:,1:].as_matrix()
test_X = testX.iloc[:,1:].as_matrix()

In [ ]:
gbm = xgb.XGBRegressor(max_depth=3, n_estimators=300, learning_rate=0.05).fit(train_X, target)

In [ ]:
importances = pd.DataFrame({'Feature':trainX.iloc[:,1:].columns, 'Importance':gbm.feature_importances_})
importances = importances.sort_values('Importance',ascending=False).set_index('Feature')
importances[0:10].iloc[::-1].plot(kind='barh',legend=False)
plt.title("XGBoost - Feature importance")
plt.show()

In [ ]:
xgb_preds = np.exp(gbm.predict(test_X))

In [ ]:
solxgb = pd.DataFrame({'Id':test['Id'], 'SalePrice':xgb_preds})
solxgb.to_csv("./solxgb.csv", index=False)
solxgb.head(3)

**Results:** This very vanilla model gave a score of 0.13175, good for 1,512th place as of this writing. I'm impressed. This is with no parameter tweaking at all, and very little feature engineering.